# PIPELINE

- torch datei vs json file vergelichen ist das möglich 
- HUggingfcae modell hinzufügen 

### Details 
1. **CLIP-Modell:** Wird verwendet, um die Wahrscheinlichkeiten zu berechnen, ob ein Bild eine Waffe enthält. Der SChwellenwert ist anpassbar, falls die Klassifikationsergebnisse unzureichend sind.
2. **Roboflow-Integration:** wird über die API initalisiert und für die Analyse von Bildern genutzt 
3. **Ergebnisvergleich:** Die Überschneidung und Unterschiede zwischen den Modellen werden als Mengenoperationen durchgeführt.
4. **Ergebnisvergleich:** Die Überschneidungen und Unterschiede zwischen den Modellen werden als Mengenoperationen durchgeführt 
5. **Batch-Verarbeitung:** Bilder werden in Blöcken von 500 verarbeitet, um Speicherplatz effizient zu nutzen. 
6. **Dynamisch Anpassung:** Falls zu wenige Bilder Klassifiziert werden, wird der Schwellenwert dynamisch abgesenkt

### 1. Laden der Daten: 
* Laden der klassifizieren Bilder mit torch-load aus der angebenen Datei -> Anmerkungen: es gibt eine json file, wo man die kennnummern und die Generierung und klassifiezierungen abspeichern kann
    + Ghet schneller als die Bilder 

* Bilder immer Blockweiese (500) Verarbeiten 
### 2. Analyse mit dem CLIP-MODELL:
* Verwendet ein CLIP-MODELL, um die Bildähnlichkeit zu bewerten und Überschneidungen zu finden. 
* Setzt einen Schwellenwert (z.B. 20%) und passe ihn dynamisch an, wenn nicht genügend Bilder Klassifiziert werden.
### 3. Analyse mit Roboflow/Huggingface-Modell:
* Führt eine weitere Analyse mit einem trainierten Roboflow-Modell (über Huggingface) durch. 
* Identifiziere Bilder mit Waffen basierend auf diesem Modell 
### 4. Überschneidungen und Statistiken:
* Vergelicht die Ergebnisse von CLIP und dem Roboflow-Modell
* Generiere eine Gegenüberstelllung der Anzhal der Bilder mit Waffen: 
    - In beiden Modellen Klassifiziert 
    - Nur in einem Modelle klassifiziert 
### 5. Bericht generieren: 
* Speichere die Ergebnisse und Statistiken in einer übersichtlichen Form (z.B. als CSV oder JSON)
* Bilder die bei beiden Matchen sollen in einer separaten liste gespeichert werden. Schnittmenge im Detail überprüfen 

### **IMPORTS**

In [3]:
import os
import torch
import numpy as np
from transformers import CLIPProcessor, CLIPModel, pipeline
from roboflow import Roboflow
import json

/Users/TheresaHartmann/opt/anaconda3/envs/env-hiwi/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'roboflow'

### **SETTINGS**

In [ ]:
CLIP_THRESHOLD = 0.2
BATCH_SIZE = 500
ROBOWFLOW_API_KEY = "your_roboflow_api_key" # Bitte Gebenfalls eigenen API verwenden, Konto notwendig
ROBOWFLOW_MODEL = "your_model_version" # TODO replace with model 

### **Roboflow INIT**

In [ ]:
rf = Roboflow(api_key=ROBOWFLOW_API_KEY)
project = rf.workspace().project("your_project_name")
roboflow_model = project.version(ROBOWFLOW_MODEL).model
# TODO: replace with pretrained model 

### **Clip INIT**

In [ ]:
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


### **METHODS**

In [ ]:
def load_images(filepath):
    # Images aus torch datei laden 
    return torch.load(filepath)

In [ ]:
def load_roboflow_results(filepath):
    # Ergebnisse aus der Roboflow datei laden und store detetcion per images 
    with open(filepath, "r") as f:
        data = json.load(f)
        
    roboflow_results = {}
    for entry in data: 
        image_name = entry["image_name"]
        detected_classes = entry["classes"]
        # List of detection weapon 
        roboflow_results[image_name] = "weapon" if detected_classes else "not weapon"

In [ ]:
def analyze_with_roboflow(images):
    "roboflow Bilder"
    weapon_images = []
    for image in images: 
        prediction = roboflow_model.predict(image, hosted = True )
        if "weapon" in prediction["predictions"]:# das muss an die Roboflow Modelle und ergebnisse angepasst werden
            weapon_images.append(image)
    return weapon_images 

In [ ]:
def analyze_with_clip(images, threshold): 
    # Analyze Bilder mit CLIP-MODELL
    results_per_threshold = {t: [] for t in threshold}
    
    for image in images:
        image_name = image.split("/")[-1]
        
        inputs = clip_processor(text=["weapon", "not weapon"],images=image, return_tenors="pt", padding= True)
        outputs = clip_model(**inputs)
        logits_per_image = outputs.logits_per_image.softmax(dim=1).detach().numpy()
        
        for t in threshold: 
            results_per_threshold[t][image_name]= "weapon" if logits_per_image[0][0] > t else "not weapon"
            
        
    return results_per_threshold

### **COMPARISONS**


In [ ]:
def compare_results(clip_results, roboflow_results):
    # Vergeliche Ergebnisse und erstelle Statistiken
    set_clip = set(clip_results)
    set_roboflow = set(roboflow_results)
    
    overlap = set_clip.intersection(set_roboflow)
    only_clip = set_clip.difference(set_roboflow)
    only_roboflow = set_roboflow.difference(set_clip)
    
    return {
        "overlap": len(overlap),
        "only_clip": len(only_clip),
        "only_roboflow": len(only_roboflow),
    }
    

In [ ]:
def compare_results_second( clip_results, roboflow_results):
    # Compare CLIP and Roboflow classifictaion per threshold
    comparison_stats = {}
    
    for t, clip_classifications in clip_results.items():
        correct = 0
        total = len(roboflow_results)
        false_positives = 0
        false_negatives = 0 
        
        for image_name, roboflow_label in roboflow_results.items():
            clip_label = clip_classifications.get(image_name, "not weapon")  # Default to "not weapon" if missing
            
            if clip_label == roboflow_label:
                correct += 1 
            elif clip_label == "weapon" and roboflow_label == "not weapon":
                false_postives += 1 
            elif clip_label == "not weapon" and roboflow_label == "weapon":
                false_negatives += 1 
                
         # Calculate precision, recall and F1 score
        precision = correct / (correct + false_positives) if ( correct + false_positives)> 0 else 0
        recall = correct / (correct + false_negatives) if (correct + false_negatives) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        comparison_stats[t] = {
            "correct": correct,
            "false_positives": false_positives,
            "false_negatives": false_negatives,
            "precision": precision,
            "recall": recall,
            "f1_score": f1_score
        }
        
    return comparison_stats           


In [ ]:
#def main(): 
#   image_filepath = "/ceph/lprasse/ClimateVisions/Tweet_Dataset/nsfw_cleaning/final_res/sim_violence_4.torch"
#   images = load_images(image_filepath)
#   
#   for i in range(0, len(images), BATCH_SIZE):
#       batch = images[i:i + BATCH_SIZE]

In [ ]:
def main():
    # Load images from torch file
    image_filepath = "/ceph/lprasse/ClimateVisions/Tweet_Dataset/nsfw_cleaning/final_res/sim_violence_3.torch"
    images = load_images(image_filepath)

    # Load Roboflow JSON results
    roboflow_results = load_roboflow_results("weapon_classification.json")

    # Define thresholds to test
    thresholds = [i * 0.1 for i in range(1, 10)]  # 0.1, 0.2, ..., 0.9

    # Get CLIP results for each threshold
    clip_results_per_threshold = analyze_with_clip(images, thresholds)

    # Compare CLIP and Roboflow results
    comparison_stats = compare_results_second(clip_results_per_threshold, roboflow_results)

    # Find the best threshold based on F1-score
    best_threshold = max(comparison_stats, key=lambda t: comparison_stats[t]["f1_score"])
    best_f1 = comparison_stats[best_threshold]["f1_score"]

    print(f"Best threshold: {best_threshold} with F1-score: {best_f1}")
    print("Comparison stats per threshold:", json.dumps(comparison_stats, indent=4))

    # Save results for later analysis
    with open("comparison_results.json", "w") as f:
        json.dump(comparison_stats, f, indent=4)
